In [58]:
# Load Libraries
import pandas as pd

In [59]:
# Viếng thăm

# Define the file path
file_path = 'Dataset/dms.visit.partner_visit.xlsx'

# Load the Excel file
df_visit = pd.read_excel(file_path)

# Group by 'Nhân viên viếng thăm' and count the number of visits
visit_counts = df_visit.groupby('Nhân viên viếng thăm').size().reset_index(name='Tổng số lượt đi')

# Display the result
visit_counts


,Nhân viên viếng thăm,Tổng số lượt đi
0,MT- MEKONG1,33
1,MT- MIENDONG,69
2,MT- MIENTRUNG,47
3,MT-QLKDMN,22
4,MT-QLKDMN3,1
5,MTHCM-KV1,85
6,MTHCM-KV2,92
7,MTHCM-KV3,105
8,MTHCM-KV5,102
9,MTHCM-KV6,75


In [60]:
# Thiết lập Data: Độ phủ

# Độ phủ


# Load data khách hàng
file_path_customer = 'Dataset/customer.xlsx'  # Update this to the correct path for df_customer
df_customer=pd.read_excel(file_path_customer,sheet_name='Data khách hàng')


# Loading: ĐỘ PHỦ

file_path_fill = 'Dataset/dms.visit.partner_fill.xlsx'
df_fill = pd.read_excel(file_path_fill)

# Loading: LISTING
file_path_listing = 'Dataset/listing.xlsx'
df_listing=pd.read_excel(file_path_listing)

# CUSTOMER -> MERGED

# Merge df_fill with df_customer to get 'Nhân viên KD' and 'Loại hình'
df_merged = pd.merge(df_fill, df_customer[['Mã', 'Nhân viên KD','Hệ thống' ,'Loại hình']], left_on='Khách hàng/Mã', right_on='Mã', how='right')


# Bỏ quản lý
list_quan_ly = ['MT-QLKDMN','MT-QLKDMN3']

df_fillmerged=df_merged[-df_merged['Nhân viên viếng thăm'].isin(list_quan_ly)].rename(columns={
    'Chi tiết kiểm tồn/Sản phẩm/Mã nội bộ': 'Mã sản phẩm',
    'Khách hàng/Mã': 'Mã khách hàng',
    'Khách hàng' : 'Mã khách hàng ERP',
    'Khách hàng/Tên':'Tên khách hàng',
    'Chi tiết kiểm tồn/Sản phẩm/Tên': 'Tên sản phẩm',
    'Chi tiết kiểm tồn/Số lượng chẵn': 'Tồn',
    'Chi tiết kiểm tồn/Sản phẩm/Đơn vị tính':'Đơn vị tính'
})


# Convert 'Đăng nhập' to datetime format if it isn't already
df_fillmerged['Đăng nhập'] = pd.to_datetime(df_fillmerged['Đăng nhập'], errors='coerce')

# Create a new column 'Date_Signin' by extracting the date from 'Đăng nhập'
df_fillmerged['Date_Signin'] = df_fillmerged['Đăng nhập'].dt.date



# TÍNH TOÁN ĐỘ PHỦ 

# Group by 'Mã khách hàng', 'Mã sản phẩm', 'Date_Signin' and calculate the sum of 'Tồn'
df_checkshow = df_fillmerged.groupby(['Mã khách hàng', 'Mã sản phẩm', 'Date_Signin']).agg({'Tồn': 'sum'}).reset_index()

# Add 'Check_Show' and 'Product_Exist' columns based on the 'Tồn' value
df_checkshow['Check_Show'] = df_checkshow['Tồn'].apply(lambda x: 1 if x > 0 else 0)


# Create df_listing with unique values and Product Exist = 1
df_listing['Product_Exist'] = 1


# Merge df_checkshow with df_listing to get all columns in df_checkshow
df_fill_listing = df_listing.merge(df_checkshow, on=['Mã khách hàng', 'Mã sản phẩm'], how='right')

# Replace NA/blank values in 'Check_Show' with 0
df_fill_listing['Check_Show'].fillna(0, inplace=True)
df_fill_listing['Product_Exist'].fillna(1, inplace=True)



# Merge df_fill_listing with df_fillmerged to get 'Nhân viên KD' and 'Loại hình'
df_fill_listing = df_fill_listing.merge(df_fillmerged[['Mã khách hàng', 'Mã sản phẩm', 'Nhân viên KD', 'Loại hình']], on=['Mã khách hàng', 'Mã sản phẩm'], how='left').drop_duplicates()

# Ensure Product_Exist is counted only once per unique 'Mã khách hàng' and 'Mã sản phẩm'
df_fill_listing = df_fill_listing.drop_duplicates(subset=['Mã khách hàng', 'Mã sản phẩm','Nhân viên KD', 'Loại hình'])

# ------------ PIVOT

# Pivot the data
df_pivot = df_fill_listing.groupby(['Nhân viên KD', 'Loại hình']).agg({'Check_Show': 'sum', 'Product_Exist': 'sum'}).reset_index()

# Calculate 'Fill_Store_percent' as Sum(Check_Show) / Sum(Product_Exist) 
df_pivot['Fill_Store_percent'] = (df_pivot['Check_Show'] / df_pivot['Product_Exist'])


# Create the pivot table
pivot_fill_store = df_pivot.pivot_table(index='Nhân viên KD', columns='Loại hình', values='Fill_Store_percent').reset_index()

# Calculate grand totals for each column
pivot_fill_store['Grand Total'] = pivot_fill_store.mean(axis=1, numeric_only=True)

# Calculate the overall grand total as the mean of the grand total column
grand_total = pivot_fill_store['Grand Total'].mean()

# Add the grand total row
pivot_fill_store.loc['Grand Total'] = pivot_fill_store.mean(numeric_only=True)
pivot_fill_store.at['Grand Total', 'Nhân viên KD'] = 'Grand Total'

# Calculate and add the overall grand total in the grand total row
pivot_fill_store.at['Grand Total', 'Grand Total'] = grand_total


pivot_fill_store


C:\Users\Nguyen Vu\AppData\Local\Temp\ipykernel_15212\3139504947.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_fill_listing['Check_Show'].fillna(0, inplace=True)
C:\Users\Nguyen Vu\AppData\Local\Temp\ipykernel_15212\3139504947.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

Loại hình,Nhân viên KD,Siêu Thị,Win+,Đại Siêu Thị,Grand Total
0,MT- MEKONG,NaN,1.000000,0.394097,0.697049
1,MT- MIENDONG,0.629630,1.000000,0.996627,0.875419
2,MTHCM-KV1,0.391230,1.000000,0.487562,0.626264
3,MTHCM-KV2,0.252590,1.000000,0.560907,0.604499
4,MTHCM-KV3,0.327846,0.857143,0.564516,0.583168
5,MTHCM-KV5,0.282120,0.550562,0.489247,0.440643
6,MTHCM-KV6,0.282297,1.000000,0.493976,0.592091
7,MTHCM-KV7,0.308134,0.796296,0.580808,0.561746
Grand Total,Grand Total,0.353407,0.900500,0.570968,0.622610


In [61]:

# VIẾT KẾT QUẢ RA EXCEL
# Export dataframes to multiple sheets in an Excel file
output_file = 'Output/output.xlsx'  # Define the output file path

with pd.ExcelWriter(output_file) as writer:
    visit_counts.to_excel(writer,sheet_name='visit',index=False)
    df_fill_listing.to_excel(writer, sheet_name='fill_listing', index=False)
    df_pivot.to_excel(writer, sheet_name='pivot', index=False)
    pivot_fill_store.to_excel(writer, sheet_name='pivot_fill_store', index=False)

print(f"Dataframes exported successfully to {output_file}")

Dataframes exported successfully to Output/output.xlsx
